In [22]:
from elasticsearch import Elasticsearch

# Create Elasticsearch client
es = Elasticsearch(
    "http://localhost:9200",  # Changed from https to http
    basic_auth=("elastic", "pass"),  # Use your actual password
)
# Update disk watermark thresholds
es.cluster.put_settings(
    body={
        "persistent": {
            "cluster.routing.allocation.disk.watermark.low": "99%",
            "cluster.routing.allocation.disk.watermark.high": "99%",
            "cluster.routing.allocation.disk.watermark.flood_stage": "99%",
        }
    }
)
# Test connection
try:
    if es.ping():
        print("Successfully connected to Elasticsearch")
        print(es.info())
    else:
        print("Could not connect to Elasticsearch")
except Exception as e:
    print(f"Connection failed: {e}")

Successfully connected to Elasticsearch
{'name': '66123632dc80', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'tFhb53A1Rw2c0dBvX5pkpA', 'version': {'number': '8.17.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b6a7fed44faa321997703718f07ee0420804b41', 'build_date': '2024-12-11T12:08:05.663969764Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [23]:
# Elasticsearch mappings for the models

USER_MAPPING = {
    "mappings": {
        "properties": {
            "email": {"type": "keyword"},
            "name": {"type": "text"},
            "password": {"type": "keyword"},
            "embedding": {
                "type": "dense_vector",
                "dims": 768,  # Adjust dimension based on your embedding size
            },
        }
    }
}

RECIPE_MAPPING = {
    "mappings": {
        "properties": {
            "id": {"type": "integer"},
            "title": {"type": "text"},
            "ingredients": {"type": "text"},
            "instructions": {"type": "text"},
            "prep_time": {"type": "integer"},
            "cook_time": {"type": "integer"},
            "cuisine": {"type": "keyword"},
            "course": {"type": "keyword"},
            "diet": {"type": "keyword"},
            "image": {"type": "keyword", "index": False},
            "url": {"type": "keyword", "index": False},
            "embedding": {
                "type": "dense_vector",
                "dims": 768,  # Adjust dimension based on your embedding size
            },
        }
    }
}

FEEDBACK_MAPPING = {
    "mappings": {
        "properties": {
            "email": {"type": "keyword"},
            "input_description": {"type": "text"},
            "input_image": {"type": "text", "index": False},
            "recipe_id": {"type": "integer"},
            "rating": {"type": "integer"},
            "comment": {"type": "text"},
        }
    }
}

USER_REVIEW_MAPPING = {
    "mappings": {
        "properties": {
            "email": {"type": "keyword"},
            "reviews": {"type": "nested", "properties": {"text": {"type": "text"}}},
        }
    }
}

RECIPE_ADD_MAPPING = {
    "mappings": {
        "properties": {
            "id": {"type": "integer"},
            "title": {"type": "text"},
            "ingredients": {"type": "text"},
            "instructions": {"type": "text"},
            "prep_time": {"type": "integer"},
            "cook_time": {"type": "integer"},
            "cuisine": {"type": "keyword"},
            "course": {"type": "keyword"},
            "diet": {"type": "keyword"},
            "image": {"type": "keyword", "index": False},
            "url": {"type": "keyword", "index": False},
            "embedding": {
                "type": "dense_vector",
                "dims": 768,  # Adjust dimension based on your embedding size
            },
            "accepted": {"type": "boolean"},
        }
    }
}

In [25]:
# ... existing elasticsearch import and client setup ...
def create_indices(es_client):
    """Create all required indices if they don't exist"""
    for index_name, mapping in MAPPINGS.items():
        try:
            if not es_client.indices.exists(index=index_name):
                print(f"Creating index '{index_name}'...")
                es_client.indices.create(index=index_name, body=mapping)
                print(f"Successfully created index '{index_name}'")
            else:
                print(f"Index '{index_name}' already exists")
        except Exception as e:
            print(f"Error creating index '{index_name}': {e}")


# Define the mapping dictionary
MAPPINGS = {
    "users": USER_MAPPING,
    "recipes": RECIPE_MAPPING,
    "feedback": FEEDBACK_MAPPING,
    "user_reviews": USER_REVIEW_MAPPING,
    "recipe_additions": RECIPE_ADD_MAPPING,
}

# Create all indices
create_indices(es)

Creating index 'users'...
Successfully created index 'users'
Creating index 'recipes'...
Successfully created index 'recipes'
Creating index 'feedback'...
Successfully created index 'feedback'
Creating index 'user_reviews'...
Successfully created index 'user_reviews'
Creating index 'recipe_additions'...
Successfully created index 'recipe_additions'


In [24]:
def delete_indices(es_client):
    """Delete all indices defined in MAPPINGS"""
    for index_name in MAPPINGS.keys():
        try:
            if es_client.indices.exists(index=index_name):
                print(f"Deleting index '{index_name}'...")
                es_client.indices.delete(index=index_name)
                print(f"Successfully deleted index '{index_name}'")
            else:
                print(f"Index '{index_name}' does not exist")
        except Exception as e:
            print(f"Error deleting index '{index_name}': {e}")


# Delete all indices
# delete_indices(es)

Deleting index 'users'...
Successfully deleted index 'users'
Deleting index 'recipes'...
Successfully deleted index 'recipes'
Deleting index 'feedback'...
Successfully deleted index 'feedback'
Deleting index 'user_reviews'...
Successfully deleted index 'user_reviews'
Deleting index 'recipe_additions'...
Successfully deleted index 'recipe_additions'


In [26]:
def check_index_stats(es_client, index_name="recipes"):
    """
    Check if an index exists and get its document count

    Args:
        es_client: AsyncElasticsearch client
        index_name: Name of the index to check

    Returns:
        bool: True if index exists and has documents, False otherwise
    """
    try:
        # Check if index exists
        if not es_client.indices.exists(index=index_name):
            print(f"Index '{index_name}' does not exist!")
            return False

        # Get document count
        stats = es_client.count(index=index_name)
        doc_count = stats["count"]

        print(f"Index '{index_name}' contains {doc_count} documents")
        return doc_count > 0

    except Exception as e:
        print(f"Error checking index: {e}")
        return False


# Usage example:
has_documents = check_index_stats(es)
if not has_documents:
    print("Index is empty! You may need to index some documents first.")

Index 'recipes' contains 0 documents
Index is empty! You may need to index some documents first.


## read data and index it to elastic


In [27]:
import pandas as pd

df = pd.read_csv("./data.csv")
df.head()

,id,title,ingredients,instructions,prep_time,cook_time,cuisine,course,diet,image,url,embedding
0,4529,lavand-e-murgh recipe - afghani chicken in yog...,['fresh pomegranate fruit kernels few garnish'...,"['to begin making the lavand-e-murgh recipe, w...",15,25,Afghan,Dinner,High Protein Non Vegetarian,NaN,https://www.archanaskitchen.com/lavand-e-murgh...,"[[-0.0026710997335612774, 0.003612738568335771..."
1,4640,afghani dhoog recipe - cucumber mint buttermil...,"['cumin powder jeera', 'curd dahi yogurt', 'sa...",['to begin making the afghani dhoog recipe - c...,10,0,Afghan,Snack,Vegetarian,NaN,http://www.archanaskitchen.com/doogh-afghani-y...,"[[-0.014779524877667427, -0.008534302935004234..."
2,5978,malida recipe (healthy whole wheat afghan sweet),"['cardamom powder elaichi', 'dates pitted fine...","['to begin making the malida recipe, tear the ...",20,20,Afghan,Snack,Vegetarian,NaN,https://www.archanaskitchen.com/malida-recipe-...,"[[-0.01772255077958107, -0.019701037555933, -0..."
3,7092,moroccan spiced millet and lentil salad recipe,"['tomato chopped', 'extra virgin olive oil', '...",['to begin making the moroccan spiced millet a...,10,20,African,Dinner,Vegetarian,NaN,https://www.archanaskitchen.com/moroccan-spice...,"[[-0.06342744827270508, -0.01326711568981409, ..."
4,6684,chickpea & date tagine recipe,"['onion', 'cumin powder jeera', 'extra virgin ...",['to begin making the chickpea & date tagine r...,15,60,African,Dinner,High Protein Vegetarian,NaN,https://www.archanaskitchen.com/chickpea-date-...,"[[-0.03216571733355522, 0.029672250151634216, ..."


In [28]:
# Add import at the top
import ast

# Convert string representation to actual array when reading the embedding
df["embedding"] = df["embedding"].apply(ast.literal_eval)

# Now when you check the embedding, it will be an actual array
print(type(df["embedding"].loc[0]))  # Should print: <class 'list'>

<class 'list'>


index one document through recipe model


In [29]:
from elasticsearch import Elasticsearch
from typing import Dict, Any
import sys
from models import Recipe

# ... existing code ...


def index_recipe_to_elastic(
    recipe: Recipe, es_client: Elasticsearch, index_name: str = "recipes"
) -> None:
    """
    Index a recipe to Elasticsearch

    Args:
        recipe: Recipe model instance
        es_client: Elasticsearch client instance
        index_name: Name of the Elasticsearch index (default: "recipes")
    """
    doc = {
        "id": recipe.id,
        "title": recipe.title,
        "ingredients": recipe.ingredients,
        "instructions": recipe.instructions,
        "prep_time": recipe.prep_time,
        "cook_time": recipe.cook_time,
        "cuisine": recipe.cuisine,
        "course": recipe.course,
        "diet": recipe.diet,
        "image": str(recipe.image) if recipe.image else None,
        "url": str(recipe.url) if recipe.url else None,
        "embedding": recipe.embedding,
    }

    es_client.index(index=index_name, id=str(recipe.id), document=doc)
    print(f"Indexed recipe {recipe.id} to Elasticsearch")

bulk index df recipes


In [30]:
# Add necessary imports
from models import Recipe
from ast import literal_eval
import numpy as np
from elasticsearch.helpers import bulk

In [31]:
# Add necessary imports
from models import Recipe
from ast import literal_eval
import numpy as np


def row_to_recipe(row):
    """Convert a DataFrame row to a Recipe object"""
    # Convert embedding to a flat list of floats
    embedding = np.array(row.embedding).flatten().tolist()
    try:
        return Recipe(
            id=row.id,
            title=row.title,
            ingredients=(
                literal_eval(row.ingredients)
                if isinstance(row.ingredients, str)
                else row.ingredients
            ),
            instructions=(
                literal_eval(row.instructions)
                if isinstance(row.instructions, str)
                else row.instructions
            ),
            prep_time=row.prep_time,
            cook_time=row.cook_time,
            cuisine=row.cuisine,
            course=row.course,
            diet=row.diet,
            image=row.image if pd.notna(row.image) else None,
            url=row.url if pd.notna(row.url) else None,
            embedding=embedding,  # Now it's a flat list of floats
        )
    except Exception as e:
        return None

In [32]:
def bulk_index_recipe_batch(df_batch, es_client, index_name="recipes"):
    """
    Convert a batch of DataFrame rows to Recipe objects and bulk index them

    Args:
        df_batch: Pandas DataFrame batch containing recipes
        es_client: Elasticsearch client instance
        index_name: Name of the Elasticsearch index
    """
    # Convert rows to Recipe objects and filter out None values
    recipes = [
        r
        for r in (row_to_recipe(row) for _, row in df_batch.iterrows())
        if r is not None
    ]

    if not recipes:
        print("No valid recipes in this batch")
        return
    # Prepare bulk indexing actions
    actions = []
    for recipe in recipes:
        # Build document with required fields
        doc = {
            "id": recipe.id,
            "title": recipe.title,
            "ingredients": recipe.ingredients,
            "instructions": recipe.instructions,
            "prep_time": recipe.prep_time,
            "cook_time": recipe.cook_time,
            "cuisine": recipe.cuisine,
            "course": recipe.course,
            "diet": recipe.diet,
        }

        # Only add optional fields if they're not None
        if recipe.image is not None:
            doc["image"] = recipe.image
        if recipe.url is not None:
            doc["url"] = recipe.url
        if recipe.embedding is not None:
            doc["embedding"] = recipe.embedding

        actions.append({"_index": index_name, "_id": str(recipe.id), "_source": doc})

    # Perform bulk indexing
    try:
        success, failed = bulk(es_client, actions, chunk_size=500, request_timeout=30)
        print(f"Successfully indexed {success} documents")
        if failed:
            print(f"Failed to index {len(failed)} documents")
    except Exception as e:
        print(f"Error during bulk indexing: {e}")


# Usage example - process in batches of 1000
batch_size = 1000
for start_idx in range(0, len(df), batch_size):
    batch = df.iloc[start_idx : start_idx + batch_size]
    bulk_index_recipe_batch(batch, es)

/tmp/ipykernel_22820/1292315588.py:48: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  success, failed = bulk(es_client, actions, chunk_size=500, request_timeout=30)


Successfully indexed 1000 documents
Successfully indexed 1000 documents
Successfully indexed 1000 documents
Successfully indexed 1000 documents
Successfully indexed 1000 documents
Successfully indexed 44 documents
